Importations

In [7]:
import pickle
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score

Préparation des données

In [10]:
# Récupération des données
arbre = pd.read_csv("Data_Arbre.csv")

# Encodage
OE = OrdinalEncoder()
LE = LabelEncoder()
arbre[['fk_stadedev']] = OE.fit_transform(arbre[['fk_stadedev']])
arbre[['fk_nomtech']] = LE.fit_transform(arbre[['fk_nomtech']])


# Sélection des données intéressantes pour l'étude
x = arbre[['haut_tronc', 'fk_stadedev', 'tronc_diam', 'haut_tot', 'fk_nomtech']].copy()
y = arbre[['age_estim']].copy()

# Normalisation
x_norm = (x - x.min()) / (x.max() - x.min())
y_norm = (y - y.min()) / (y.max() - y.min())

# Division de la base de données
x_train, x_test, y_train, y_test = train_test_split(x_norm, y_norm, train_size=0.8, random_state=42)

# Reshape y_train and y_test
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

/Users/no/Library/CloudStorage/OneDrive-yncréa/Pro/Ecole/Cours/A3/PROJET/Projet_IA_2024/monenv/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Columns must be same length as key

Random Forest

In [5]:
# Détermination des paramètres à tester pour le RandomForest
RF_param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Création du modèle
RF = RandomForestRegressor()

# Utilisation de GridSearch pour déterminer les meilleurs paramètres à utiliser pour ce modèle
RF_GS = GridSearchCV(estimator=RF, param_grid=RF_param_grid, cv=3, n_jobs=-1, verbose=2)

# Entraînement du GridSearch
RF_GS.fit(x_train, y_train)

# Récupération et affichage des meilleurs paramètres
RF_best_params = RF_GS.best_params_

# Récupération du meilleur estimateur et prédiction
best_RF = RF_GS.best_estimator_
RF_y_pred = best_RF.predict(x_test)


Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_

/Users/no/Library/CloudStorage/OneDrive-yncréa/Pro/Ecole/Cours/A3/PROJET/Projet_IA_2024/monenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
432 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
178 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/no/Library/CloudStorage/OneDrive-yncréa/Pro/Ecole/Cours/A3/PROJET/Projet_IA_2024/monenv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/no/Library/CloudStorage/OneDrive-yncréa/Pro/Ecole/Cours/A3/PROJET/Projet_IA_2024/monenv/lib/python3.9/site-packages/sklearn/base

In [6]:
RF_mse = mean_squared_error(y_test, RF_y_pred)
RF_r2 = r2_score(y_test, RF_y_pred)
print("RF Best parameters found: ", RF_best_params)
print(f"Random Forest -> Mean Squared Error: {RF_mse}, R^2 Score: {RF_r2}")

RF Best parameters found:  {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Random Forest -> Mean Squared Error: 0.0017125504068275829, R^2 Score: 0.8203987224632799


CART

In [ ]:
# Détermination des paramètres à tester pour CART
CART_param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 30, 40],
    'min_impurity_decrease': [0.0, 0.01, 0.1]
}

# Création du modèle
DTR = DecisionTreeRegressor()

# Utilisation de GridSearch pour déterminer les meilleurs paramètres à utiliser pour ce modèle
CART_GS = GridSearchCV(estimator=DTR, param_grid=CART_param_grid, cv=3, n_jobs=-1, verbose=2)

# Entraînement GridSearch
CART_GS.fit(x_train, y_train)

# Récupération et affichage des meilleurs paramètres
CART_best_params = CART_GS.best_params_

# Récupération du meilleur estimateur et prédiction
best_CART = CART_GS.best_estimator_
CART_y_pred = best_CART.predict(x_test)


In [ ]:
CART_mse = mean_squared_error(y_test, CART_y_pred)
CART_r2 = r2_score(y_test, CART_y_pred)
print("CART Best parameters found: ", CART_best_params)
print(f"DecisionTreeRegression -> Mean Squared Error: {CART_mse}, R^2 Score: {CART_r2}")

GradientBoostingRegressor

In [ ]:
# Détermination des paramètres à tester pour GBR
GBR_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

# Création du modèle
GBR = GradientBoostingRegressor()

# Utilisation de GridSearch pour déterminer les meilleurs paramètres à utiliser pour ce modèle
GBR_GS = GridSearchCV(estimator=GBR, param_grid=GBR_param_grid, cv=3, n_jobs=-1, verbose=2)

# Entraînement GridSearch
GBR_GS.fit(x_train, y_train)

# Récupération et affichage des meilleurs paramètres
GBR_best_params = GBR_GS.best_params_

# Récupération du meilleur estimateur et prédiction
best_GBR = GBR_GS.best_estimator_
GBR_y_pred = best_GBR.predict(x_test)


In [ ]:
GBR_mse = mean_squared_error(y_test, GBR_y_pred)
GBR_r2 = r2_score(y_test, GBR_y_pred)
print("GBR Best parameters found: ", GBR_best_params)
print(f"GradientBoostingRegressor -> Mean Squared Error: {GBR_mse}, R^2 Score: {GBR_r2}")